#**Python Program to Allocate Proportional Representation (PR) Seats Using the 2021 Veracruz Election Data**


####**The main goal of this program is to compute and allocate the 20 proportional representation (PR) seats in accordance with the procedure established in articles 247 to 249 of the Electoral Code for the State of Veracruz**

###**1.- Import the libraries to be used**

In [194]:
#Import libraries
import pandas as pd
import os
import numpy as np
from google.colab import drive
import warnings
warnings.filterwarnings('ignore')
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [195]:
#Import Veracruz data 2021 election
root_path = "drive/MyDrive/RP_Veracruz_2021"
orig = root_path + "/original_db"
os.listdir(orig)

['RP_veracruz_2021.csv']

###**2.- Generating a dataframe with the 2021 voting data**

In [196]:
#Creating a dataframe with the 2021 voting data
df_rp = pd.read_csv(orig + "/RP_veracruz_2021.csv")
#Computing the total voting
total_votos = df_rp['Votación'].sum()
#Adding the rows with the sum of the total vote to the dataframe
df_rp.loc[len(df_rp.index)] = ['Votación total', total_votos]
df_rp

,Partido,Votación
0,PAN,542116
1,PRI,367436
2,PRD,204365
3,VERDE,213508
4,PT,129147
5,MC,255904
6,MORENA,1311203
7,TxV,63903
8,PODEMOS,66414
9,P. CARDENISTA,27509


###**3.- Creating a filter for the valid vote cast and calculated the 3% threshold according to the Electoral Law of Veracruz.**

In [197]:
#Computing the valid vote cast by subtracting invalid votes (VN) and non-registered candidates (CNR)
votos_nulos = df_rp.loc[df_rp['Partido'] == 'VN', 'Votación'].values[0]
candidatos_no_registrados = df_rp.loc[df_rp['Partido'] == 'CNR', 'Votación'].values[0]
votacion_valida_emitida = total_votos - votos_nulos - candidatos_no_registrados
#Adding the row with the valid vote cast to the dataframe
df_rp.loc[len(df_rp.index)] = ['Votación válida emitida', votacion_valida_emitida]
#Creating a dataframe filter to exclude CNR, VN and Total Voting
part_vve = df_rp[~df_rp['Partido'].isin(['CNR', 'VN', 'Votación total'])]
#Filtered dataframe
part_vve

,Partido,Votación
0,PAN,542116
1,PRI,367436
2,PRD,204365
3,VERDE,213508
4,PT,129147
5,MC,255904
6,MORENA,1311203
7,TxV,63903
8,PODEMOS,66414
9,P. CARDENISTA,27509


In [198]:
#Computing the 3% threshold to determine those parties that meet this criterion
umbral_3 = votacion_valida_emitida * 0.03

###**4.- Computing the valid state vote cast for those parties that meet the 3% threshold**

In [199]:
#Filtering parties that meet the 3% of valid vote cast
part_veve = df_rp[(df_rp['Votación'] >= umbral_3) & ~df_rp['Partido'].isin(['CNR', 'VN', 'Votación total', 'Votación válida emitida'])]
#Computing the sum of the vote of the parties that meet the 3% threshold
votacion_estatal_valida_emitida = part_veve['Votación'].sum()
#Adding the row with the valid state vote cast to the filtered dataframe
part_veve.loc[len(part_veve.index)] = ['Votación estatal válida emitida', votacion_estatal_valida_emitida]
part_veve

,Partido,Votación
0,PAN,542116
1,PRI,367436
2,PRD,204365
3,VERDE,213508
4,PT,129147
5,MC,255904
6,MORENA,1311203
13,FxM,110605
8,Votación estatal válida emitida,3134284


###**5.- Computing the natural quotient according to Veracruz law by dividing the valid state vote cast by the total number of PR seats to be distributed**

In [200]:
#Total number of seats to be distributed according to Veracruz electoral law
total_escanos = 20
#Computing natural quotien
cociente_natural = votacion_estatal_valida_emitida / total_escanos
cociente_natural_rounded = round(cociente_natural, 1)
print(f'Cociente natural: {cociente_natural_rounded}')

Cociente natural: 156714.2


In [201]:
#Eliminating the row Valid state vote cast
part_veve = part_veve[part_veve['Partido'] != 'Votación estatal válida emitida']
part_veve

,Partido,Votación
0,PAN,542116
1,PRI,367436
2,PRD,204365
3,VERDE,213508
4,PT,129147
5,MC,255904
6,MORENA,1311203
13,FxM,110605


###**6.- Step 1: Allocation by natural quotient (NC)**

In [202]:
#Step 1: Allocating by natural quotient (NC)
part_veve['Escanos_CN'] = (part_veve['Votación'] // cociente_natural).astype(int)
#Adding the column of the natural quotient for printing
part_veve['Cociente_Natural'] = cociente_natural_rounded
#Printing results by natural quotient excluding valid state vote cast
part_veve[['Partido', 'Votación', 'Cociente_Natural', 'Escanos_CN']]
#We can see that 16 seats are allocated by dividing the state vote cast and the natural quotient

,Partido,Votación,Cociente_Natural,Escanos_CN
0,PAN,542116,156714.2,3
1,PRI,367436,156714.2,2
2,PRD,204365,156714.2,1
3,VERDE,213508,156714.2,1
4,PT,129147,156714.2,0
5,MC,255904,156714.2,1
6,MORENA,1311203,156714.2,8
13,FxM,110605,156714.2,0


###**7. Step 2: Allocation by the largest remainder method (LR)**

In [203]:
# Step 2: Allocating by the largest remainder method (Resto Mayor)
# Number of seats remaining: 4 seats
Escanos_RM = total_escanos - part_veve['Escanos_CN'].sum()
print(f'Escanos RM: {Escanos_RM}')

Escanos RM: 4


In [204]:
#Computing the votes used and the largest remainder method
part_veve['Votos_utilizados'] = part_veve['Escanos_CN'] * cociente_natural
part_veve['Resto_mayor'] = part_veve['Votación'] - part_veve['Votos_utilizados']
#We sort the parties by the largest remainder method in descending order
part_veve = part_veve.sort_values(by='Resto_mayor', ascending=False)
#Allocating the remaining seats to the parties with the largest remainder
part_veve['Escanos_RM'] = 0
part_veve.iloc[:Escanos_RM, part_veve.columns.get_loc('Escanos_RM')] = 1
#Adding the NC seats and the LR seats to obtain the final allocation of the 20 seats
part_veve['Escanos_totales'] = part_veve['Escanos_CN'] + part_veve['Escanos_RM']
#Printing the final DataFrame with total seats allocated
part_veve_tot = part_veve[['Partido', 'Votos_utilizados', 'Resto_mayor', 'Escanos_CN', 'Escanos_RM', 'Escanos_totales']]
part_veve_tot.loc['Total'] = part_veve_tot[['Votos_utilizados', 'Resto_mayor', 'Escanos_CN', 'Escanos_RM', 'Escanos_totales']].sum()
part_veve_tot = part_veve_tot.replace({np.nan: ''})
part_veve_tot

,Partido,Votos_utilizados,Resto_mayor,Escanos_CN,Escanos_RM,Escanos_totales
4,PT,0.0,129147.0,0.0,1.0,1.0
13,FxM,0.0,110605.0,0.0,1.0,1.0
5,MC,156714.2,99189.8,1.0,1.0,2.0
0,PAN,470142.6,71973.4,3.0,1.0,4.0
6,MORENA,1253713.6,57489.4,8.0,0.0,8.0
3,VERDE,156714.2,56793.8,1.0,0.0,1.0
1,PRI,313428.4,54007.6,2.0,0.0,2.0
2,PRD,156714.2,47650.8,1.0,0.0,1.0
Total,,2507427.2,626856.8,16.0,4.0,20.0
